In [1]:
from dask.distributed import LocalCluster, Client
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import xarray as xr
import rioxarray as rio
import utils
import velocity_utils

In [2]:
geo = shapely.Point(-49.87464811308709, 64.12407418597203)
geo = utils.misc.shapely_reprojector(geo, 4326, 3413)

In [7]:
fc = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -49.87464811308709,
          64.12407418597203
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -44.976864792026305,
          60.94853468118745
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -45.1296529031047,
          61.45323580992172
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -48.123200805234006,
          61.524767537649126
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -50.507448152078666,
          63.54389081032704
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -50.594122504225396,
          65.23192621153282
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -50.210625385444786,
          67.18629307100622
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -27.681746525891924,
          81.02195379488768
        ],
        "type": "Point"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -53.4567981022488,
          72.37831583836262
        ],
        "type": "Point"
      }
    }
  ]
}

In [16]:
vs = []
gdf = gpd.GeoDataFrame.from_features(fc, crs=4326)

for row in gdf.itertuples():
    geo = utils.misc.shapely_reprojector(row.geometry,
                                         4326,
                                         3413)
    v = velocity_utils.CentreLiner(geo=geo,
                                   buff_dist=10_000,
                                   index=row.Index,
                                   filter_cube=True,
                                   get_rgb=False,
                                   get_robust_trend=False,
                                   get_annual_median=True,
                                   sample_centreline=False
                                   )
    vs.append(v)

getting cubes from itslive
filtering velocity cube:                ddt:('335d', '395d')                    mad: 5
generating annual median field
generating stream line
getting cubes from itslive
filtering velocity cube:                ddt:('335d', '395d')                    mad: 5
generating annual median field
generating stream line
getting cubes from itslive
filtering velocity cube:                ddt:('335d', '395d')                    mad: 5
generating annual median field
generating stream line
getting cubes from itslive
filtering velocity cube:                ddt:('335d', '395d')                    mad: 5
generating annual median field
generating stream line
getting cubes from itslive
filtering velocity cube:                ddt:('335d', '395d')                    mad: 5
generating annual median field
generating stream line
getting cubes from itslive
filtering velocity cube:                ddt:('335d', '395d')                    mad: 5
generating annual median field
generating stre

In [22]:
lines = []
for v in vs:
    lines.append(v.tidy_stream)

line_gdf = gpd.GeoDataFrame(geometry=lines, crs=3413)

In [26]:
line_gdf['where'] = 'greenland'
line_gdf['type'] = 'land'
line_gdf['name'] = None

In [28]:
line_gdf.to_file('../data/land_streams.geojson', driver='GeoJSON')